In [6]:

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!pip install -q findspark


In [7]:
!pip install graphframes

In [ ]:
#download the jar file 
# wget http://dl.bintray.com/spark-packages/maven/graphframes/graphframes/0.3.0-spark2.0-s_2.11/graphframes-0.3.0-spark2.0-s_2.11.jar

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!curl -L -o "/usr/local/lib/python3.6/dist-packages/pyspark/jars/graphframes-0.8.0-spark2.4-s_2.11.jar" http://dl.bintray.com/spark-packages/maven/graphframes/graphframes/0.8.0-spark2.4-s_2.11/graphframes-0.8.0-spark2.4-s_2.11.jar

In [ ]:
#!rm -rf spark-3.0.1-bin-hadoop3.2/

In [ ]:
!ls

drive  sample_data  spark-3.0.1-bin-hadoop3.2  spark-3.0.1-bin-hadoop3.2.tgz


In [9]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell"

#/content/spark-3.0.1-bin-hadoop3.2/jars/graphframes-0.8.1-spark3.0-s_2.12.jar

In [10]:
import findspark
findspark.init()

In [11]:
import pyspark
from graphframes import *

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
#spark = SparkSession.builder.appName('fun').getOrCreate()
vertices = spark.createDataFrame([('1', 'Carter', 'Derrick', 50), 
                                  ('2', 'May', 'Derzrick', 26),
                                 ('3', 'Mills', 'Jeff', 80),
                                  ('4', 'Hood', 'Robert', 65),
                                  ('5', 'Banks', 'Mike', 93),
                                 ('98', 'Berg', 'Tim', 28),
                                 ('99', 'Page', 'Allan', 16)],
                                 ['id', 'name', 'firstname', 'age'])
edges = spark.createDataFrame([('1', '2', 'friend'), 
                               ('2', '1', 'friend'),
                              ('3', '1', 'friend'),
                              ('1', '3', 'friend'),
                               ('2', '3', 'follows'),
                               ('3', '4', 'friend'),
                               ('4', '3', 'friend'),
                               ('5', '3', 'friend'),
                               ('3', '5', 'friend'),
                               ('4', '5', 'follows'),
                              ('98', '99', 'friend'),
                              ('99', '98', 'friend')],
                              ['src', 'dst', 'type'])

g = GraphFrame(vertices,edges)

In [ ]:
results = g.pageRank(resetProbability=0.15, maxIter=2)
results.vertices.sort("pagerank", ascending=False).show()


+---+------+---------+---+------------------+
| id|  name|firstname|age|          pagerank|
+---+------+---------+---+------------------+
|  3| Mills|     Jeff| 80|1.6729166666666664|
|  1|Carter|  Derrick| 50|1.0389583333333334|
| 98|  Berg|      Tim| 28|               1.0|
| 99|  Page|    Allan| 16|               1.0|
|  5| Banks|     Mike| 93|           0.97875|
|  4|  Hood|   Robert| 65|0.7945833333333333|
|  2|   May| Derzrick| 26|0.5147916666666666|
+---+------+---------+---+------------------+



In [ ]:
results

# Reading the Metadata Json

In [13]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
import json 
from pyspark.sql import SparkSession

In [14]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
metadata = spark.read.json('/content/drive/My Drive/arXiv/arxiv-metadata-oai-snapshot.json')

In [ ]:
metadata.printSchema()

root
 |-- abstract: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- authors_parsed: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- categories: string (nullable = true)
 |-- comments: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- id: string (nullable = true)
 |-- journal-ref: string (nullable = true)
 |-- license: string (nullable = true)
 |-- report-no: string (nullable = true)
 |-- submitter: string (nullable = true)
 |-- title: string (nullable = true)
 |-- update_date: string (nullable = true)
 |-- versions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- created: string (nullable = true)
 |    |    |-- version: string (nullable = true)



In [ ]:
metadata.take(1)

[Row(abstract='  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are shown for\ndistributions of diphoton pairs produced at the energy of the Large Hadron\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\nboson are contrasted with those produced from QCD processes at the LHC, showing\nthat enhanced sensitivity to the signal can be obtained with judicious\nselection 

In [ ]:
print('The number of papers in ArXiv metadata is {}'.format(metadata.count()) ) 

The number of papers in ArXiv metadata is 1767485


In [ ]:
metadata.show(2)

+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+--------------+--------------------+-----------+--------------------+
|            abstract|             authors|      authors_parsed|   categories|            comments|                 doi|       id|         journal-ref|             license|       report-no|     submitter|               title|update_date|            versions|
+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+--------------+--------------------+-----------+--------------------+
|  A fully differe...|C. Bal\'azs, E. L...|[[Balázs, C., ], ...|       hep-ph|37 pages, 15 figu...|10.1103/PhysRevD....|0704.0001|Phys.Rev.D76:0130...|                null|ANL-HEP-PR-07-12|Pavel Nadolsky|Calculation of pr..

In [ ]:
metadata.sort("update_date").show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+-------+---------+----------------+--------------------+-----------+--------------------+
|            abstract|             authors|      authors_parsed|          categories|            comments|                 doi|       id|         journal-ref|license|report-no|       submitter|               title|update_date|            versions|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+-------+---------+----------------+--------------------+-----------+--------------------+
|  This is a suppl...|      Vasily Ogryzko|[[Ogryzko, Vasily...|q-bio.PE q-bio.CB...|             5 pages|                null|0704.0034|                null|   null|     null|Vasily Ogryzko V|Origin of adaptiv...| 2007-05-23|[[Sat, 31 Mar 200...|
|  Inter

In [ ]:
df_null = metadata.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in metadata.columns])

In [ ]:
df_agg.show() 

NameError: ignored

In [15]:
sc1 = pyspark.SparkContext.getOrCreate();
lines = sc1.textFile('/content/drive/My Drive/arXiv/arxiv-metadata-oai-snapshot.json')

In [16]:
lines.take(1)

['{"id":"0704.0001","submitter":"Pavel Nadolsky","authors":"C. Bal\\\\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan","title":"Calculation of prompt diphoton production cross sections at Tevatron and\\n  LHC energies","comments":"37 pages, 15 figures; published version","journal-ref":"Phys.Rev.D76:013009,2007","doi":"10.1103/PhysRevD.76.013009","report-no":"ANL-HEP-PR-07-12","categories":"hep-ph","license":null,"abstract":"  A fully differential calculation in perturbative quantum chromodynamics is\\npresented for the production of massive photon pairs at hadron colliders. All\\nnext-to-leading order perturbative contributions from quark-antiquark,\\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\\nall-orders resummation of initial-state gluon radiation valid at\\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\\nspecified in which the calculation is most reliable. Good agreement is\\ndemonstrated with data from the Fermilab Tevatro

In [17]:
records=lines.map(lambda x:json.loads(x))

In [ ]:
records?

In [ ]:
print("Number of Records in ArXiv Data is ",records.count())

In [18]:
category_map = {'astro-ph': 'Astrophysics',
                'astro-ph.CO': 'Cosmology and Nongalactic Astrophysics',
                'astro-ph.EP': 'Earth and Planetary Astrophysics',
                'astro-ph.GA': 'Astrophysics of Galaxies',
                'astro-ph.HE': 'High Energy Astrophysical Phenomena',
                'astro-ph.IM': 'Instrumentation and Methods for Astrophysics',
                'astro-ph.SR': 'Solar and Stellar Astrophysics',
                'cond-mat.dis-nn': 'Disordered Systems and Neural Networks',
                'cond-mat.mes-hall': 'Mesoscale and Nanoscale Physics',
                'cond-mat.mtrl-sci': 'Materials Science',
                'cond-mat.other': 'Other Condensed Matter',
                'cond-mat.quant-gas': 'Quantum Gases',
                'cond-mat.soft': 'Soft Condensed Matter',
                'cond-mat.stat-mech': 'Statistical Mechanics',
                'cond-mat.str-el': 'Strongly Correlated Electrons',
                'cond-mat.supr-con': 'Superconductivity',
                'cs.AI': 'Artificial Intelligence',
                'cs.AR': 'Hardware Architecture',
                'cs.CC': 'Computational Complexity',
                'cs.CE': 'Computational Engineering, Finance, and Science',
                'cs.CG': 'Computational Geometry',
                'cs.CL': 'Computation and Language',
                'cs.CR': 'Cryptography and Security',
                'cs.CV': 'Computer Vision and Pattern Recognition',
                'cs.CY': 'Computers and Society',
                'cs.DB': 'Databases',
                'cs.DC': 'Distributed, Parallel, and Cluster Computing',
                'cs.DL': 'Digital Libraries',
                'cs.DM': 'Discrete Mathematics',
                'cs.DS': 'Data Structures and Algorithms',
                'cs.ET': 'Emerging Technologies',
                'cs.FL': 'Formal Languages and Automata Theory',
                'cs.GL': 'General Literature',
                'cs.GR': 'Graphics',
                'cs.GT': 'Computer Science and Game Theory',
                'cs.HC': 'Human-Computer Interaction',
                'cs.IR': 'Information Retrieval',
                'cs.IT': 'Information Theory',
                'cs.LG': 'Machine Learning',
                'cs.LO': 'Logic in Computer Science',
                'cs.MA': 'Multiagent Systems',
                'cs.MM': 'Multimedia',
                'cs.MS': 'Mathematical Software',
                'cs.NA': 'Numerical Analysis',
                'cs.NE': 'Neural and Evolutionary Computing',
                'cs.NI': 'Networking and Internet Architecture',
                'cs.OH': 'Other Computer Science',
                'cs.OS': 'Operating Systems',
                'cs.PF': 'Performance',
                'cs.PL': 'Programming Languages',
                'cs.RO': 'Robotics',
                'cs.SC': 'Symbolic Computation',
                'cs.SD': 'Sound',
                'cs.SE': 'Software Engineering',
                'cs.SI': 'Social and Information Networks',
                'cs.SY': 'Systems and Control',
                'econ.EM': 'Econometrics',
                'eess.AS': 'Audio and Speech Processing',
                'eess.IV': 'Image and Video Processing',
                'eess.SP': 'Signal Processing',
                'gr-qc': 'General Relativity and Quantum Cosmology',
                'hep-ex': 'High Energy Physics - Experiment',
                'hep-lat': 'High Energy Physics - Lattice',
                'hep-ph': 'High Energy Physics - Phenomenology',
                'hep-th': 'High Energy Physics - Theory',
                'math.AC': 'Commutative Algebra',
                'math.AG': 'Algebraic Geometry',
                'math.AP': 'Analysis of PDEs',
                'math.AT': 'Algebraic Topology',
                'math.CA': 'Classical Analysis and ODEs',
                'math.CO': 'Combinatorics',
                'math.CT': 'Category Theory',
                'math.CV': 'Complex Variables',
                'math.DG': 'Differential Geometry',
                'math.DS': 'Dynamical Systems',
                'math.FA': 'Functional Analysis',
                'math.GM': 'General Mathematics',
                'math.GN': 'General Topology',
                'math.GR': 'Group Theory',
                'math.GT': 'Geometric Topology',
                'math.HO': 'History and Overview',
                'math.IT': 'Information Theory',
                'math.KT': 'K-Theory and Homology',
                'math.LO': 'Logic',
                'math.MG': 'Metric Geometry',
                'math.MP': 'Mathematical Physics',
                'math.NA': 'Numerical Analysis',
                'math.NT': 'Number Theory',
                'math.OA': 'Operator Algebras',
                'math.OC': 'Optimization and Control',
                'math.PR': 'Probability',
                'math.QA': 'Quantum Algebra',
                'math.RA': 'Rings and Algebras',
                'math.RT': 'Representation Theory',
                'math.SG': 'Symplectic Geometry',
                'math.SP': 'Spectral Theory',
                'math.ST': 'Statistics Theory',
                'math-ph': 'Mathematical Physics',
                'nlin.AO': 'Adaptation and Self-Organizing Systems',
                'nlin.CD': 'Chaotic Dynamics',
                'nlin.CG': 'Cellular Automata and Lattice Gases',
                'nlin.PS': 'Pattern Formation and Solitons',
                'nlin.SI': 'Exactly Solvable and Integrable Systems',
                'nucl-ex': 'Nuclear Experiment',
                'nucl-th': 'Nuclear Theory',
                'physics.acc-ph': 'Accelerator Physics',
                'physics.ao-ph': 'Atmospheric and Oceanic Physics',
                'physics.app-ph': 'Applied Physics',
                'physics.atm-clus': 'Atomic and Molecular Clusters',
                'physics.atom-ph': 'Atomic Physics',
                'physics.bio-ph': 'Biological Physics',
                'physics.chem-ph': 'Chemical Physics',
                'physics.class-ph': 'Classical Physics',
                'physics.comp-ph': 'Computational Physics',
                'physics.data-an': 'Data Analysis, Statistics and Probability',
                'physics.ed-ph': 'Physics Education',
                'physics.flu-dyn': 'Fluid Dynamics',
                'physics.gen-ph': 'General Physics',
                'physics.geo-ph': 'Geophysics',
                'physics.hist-ph': 'History and Philosophy of Physics',
                'physics.ins-det': 'Instrumentation and Detectors',
                'physics.med-ph': 'Medical Physics',
                'physics.optics': 'Optics',
                'physics.plasm-ph': 'Plasma Physics',
                'physics.pop-ph': 'Popular Physics',
                'physics.soc-ph': 'Physics and Society',
                'physics.space-ph': 'Space Physics',
                'q-bio.BM': 'Biomolecules',
                'q-bio.CB': 'Cell Behavior',
                'q-bio.GN': 'Genomics',
                'q-bio.MN': 'Molecular Networks',
                'q-bio.NC': 'Neurons and Cognition',
                'q-bio.OT': 'Other Quantitative Biology',
                'q-bio.PE': 'Populations and Evolution',
                'q-bio.QM': 'Quantitative Methods',
                'q-bio.SC': 'Subcellular Processes',
                'q-bio.TO': 'Tissues and Organs',
                'q-fin.CP': 'Computational Finance',
                'q-fin.EC': 'Economics',
                'q-fin.GN': 'General Finance',
                'q-fin.MF': 'Mathematical Finance',
                'q-fin.PM': 'Portfolio Management',
                'q-fin.PR': 'Pricing of Securities',
                'q-fin.RM': 'Risk Management',
                'q-fin.ST': 'Statistical Finance',
                'q-fin.TR': 'Trading and Market Microstructure',
                'quant-ph': 'Quantum Physics',
                'stat.AP': 'Applications',
                'stat.CO': 'Computation',
                'stat.ME': 'Methodology',
                'stat.ML': 'Machine Learning',
                'stat.OT': 'Other Statistics',
                'stat.TH': 'Statistics Theory'}

In [19]:
list_of_interest = ('stat', 'math', 'cs', 'econ')
keys = list(category_map.keys())
categories = [i for i in keys if i.startswith(list_of_interest)]

docs = (records.filter(lambda x:any(ele in x['categories'] for ele in categories)==True))

docs 

PythonRDD[5] at RDD at PythonRDD.scala:53

In [20]:
get_metadata = lambda x: {'id': x['id'].replace('\n',' '),
                  'title': x['title'].replace('\n',' '),
                  'authors': x['authors'].replace('\n',' '),
                  'authors_parsed' : x['authors_parsed'],
                  'category':x['categories'].replace('\n',' '),
                  'abstract':x['abstract'].replace('\n',' '),
                 'version':x['versions'][-1]['created'].replace('\n',' ')}

In [21]:
filterddocs = docs.map(get_metadata)

In [22]:
spark = SparkSession.builder.getOrCreate()

In [23]:
df = filterddocs.toDF()

/content/spark-3.0.1-bin-hadoop3.2/python/pyspark/sql/session.py:401: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [24]:
df.show() 

+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+
|            abstract|             authors|      authors_parsed|            category|       id|               title|             version|
+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+
|  We describe a n...|Ileana Streinu an...|[[Streinu, Ileana...|       math.CO cs.CG|0704.0002|Sparsity-certifyi...|Sat, 13 Dec 2008 ...|
|  We show that a ...|        David Callan| [[Callan, David, ]]|             math.CO|0704.0004|A determinant of ...|Sat, 31 Mar 2007 ...|
|  In this paper w...|Wael Abu-Shammala...|[[Abu-Shammala, W...|     math.CA math.FA|0704.0005|From dyadic $\Lam...|Mon, 2 Apr 2007 1...|
|  Partial cubes a...|  Sergei Ovchinnikov|[[Ovchinnikov, Se...|             math.CO|0704.0010|Partial cubes: st...|Sat, 31 Mar 2007 ...|
|  In this paper w...|Clifton Cunn

In [25]:
cols = ['id','abstract']
abstractdf = df.select(*cols)

abstractdf.show(2)
abstractdf.count() 

+---------+--------------------+
|       id|            abstract|
+---------+--------------------+
|0704.0002|  We describe a n...|
|0704.0004|  We show that a ...|
+---------+--------------------+
only showing top 2 rows



KeyboardInterrupt: ignored

# Cosine Similarity



In [26]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, NGram, HashingTF, MinHashLSH,StopWordsRemover
import regex as re 

In [27]:
filter_columns= lambda x: {'id': x['id'].replace('\n',' '),
                  'abstract':x['abstract'].replace('\n',' ')}

In [28]:
rdd = docs.map(filter_columns)

In [29]:
rdd.take(1)

[{'abstract': '  We describe a new algorithm, the $(k,\\ell)$-pebble game with colors, and use it obtain a characterization of the family of $(k,\\ell)$-sparse graphs and algorithmic solutions to a family of problems concerning tree decompositions of graphs. Special instances of sparse graphs appear in rigidity theory and have received increased attention in recent years. In particular, our colored pebbles generalize and strengthen the previous results of Lee and Streinu and give a new proof of the Tutte-Nash-Williams characterization of arboricity. We also present a new decomposition that certifies sparsity based on the $(k,\\ell)$-pebble game with colors. Our work also exposes connections between pebble game algorithms and previous sparse graph algorithms by Gabow, Gabow and Westermann and Hendrickson. ',
  'id': '0704.0002'}]

In [ ]:
#clean text 

In [30]:
df.show(1)

+--------------------+--------------------+--------------------+-------------+---------+--------------------+--------------------+
|            abstract|             authors|      authors_parsed|     category|       id|               title|             version|
+--------------------+--------------------+--------------------+-------------+---------+--------------------+--------------------+
|  We describe a n...|Ileana Streinu an...|[[Streinu, Ileana...|math.CO cs.CG|0704.0002|Sparsity-certifyi...|Sat, 13 Dec 2008 ...|
+--------------------+--------------------+--------------------+-------------+---------+--------------------+--------------------+
only showing top 1 row



In [31]:
def clean_text(c):
  c = lower(c)
  c = regexp_replace(c, "^rt ", "")
  c = regexp_replace(c, "(https?\://)\S+", "")
  c = regexp_replace(c, "[^a-zA-Z0-9\\s]", "")
  return c

In [32]:
df = rdd.toDF(["abstract", "ID"])
clean_text_df = df.select('ID',clean_text(col("abstract")).alias("abstract"))
clean_text_df = clean_text_df.withColumn('abstract', trim(clean_text_df.abstract))

/content/spark-3.0.1-bin-hadoop3.2/python/pyspark/sql/session.py:401: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [33]:
clean_text_df.show() 

+---------+--------------------+
|       ID|            abstract|
+---------+--------------------+
|0704.0002|we describe a new...|
|0704.0004|we show that a de...|
|0704.0005|in this paper we ...|
|0704.0010|partial cubes are...|
|0704.0011|in this paper we ...|
|0704.0012|recently bruinier...|
|0704.0013|serre obtained th...|
|0704.0014|in this article w...|
|0704.0019|in this note we g...|
|0704.0022|we present lie gr...|
|0704.0026|zerodivisors zds ...|
|0704.0028|we prove pfaffian...|
|0704.0040|as in the cases o...|
|0704.0041|we formulate a qu...|
|0704.0046|in a quantum mech...|
|0704.0047|the intelligent a...|
|0704.0049|we present an alg...|
|0704.0050|part i describes ...|
|0704.0053|the aim of the pr...|
|0704.0054|in this paper we ...|
+---------+--------------------+
only showing top 20 rows



In [34]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="abstract", outputCol="words")
tokenized = tokenizer.transform(clean_text_df)

In [ ]:
tokenized.show(2)

+---------+--------------------+--------------------+
|       ID|            abstract|               words|
+---------+--------------------+--------------------+
|0704.0002|we describe a new...|[we, describe, a,...|
|0704.0004|we show that a de...|[we, show, that, ...|
+---------+--------------------+--------------------+
only showing top 2 rows



In [35]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
cleaned_df = remover.transform(tokenized)
#cleaned_df = remover.transform(tokenized).show(truncate=False)

In [36]:
cleaned_df.show(truncate=False)

+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [37]:
from pyspark.ml.feature import HashingTF, IDF

In [38]:
hashingTF = HashingTF(inputCol="words", outputCol="tf")
tf = hashingTF.transform(cleaned_df)

idf = IDF(inputCol="tf", outputCol="feature").fit(tf)
tfidf = idf.transform(tf)

In [39]:
tfidf.show(truncate=False)

+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [40]:
#Compute L2 norm:
from pyspark.ml.feature import Normalizer
normalizer = Normalizer(inputCol="feature", outputCol="norm")
data = normalizer.transform(tfidf)

In [59]:
test = data.limit(10)

dot_udf = F.udf(lambda x,y: float(x.dot(y)), DoubleType())

result2 = test.alias("i").join(test.alias("j"), F.col("i.ID") < F.col("j.ID"))\
    .select(
        F.col("i.ID").alias("i"), 
        F.col("j.ID").alias("j"), 
        dot_udf("i.norm", "j.norm").alias("dot"))\
    .sort("i", "j")

result2 = result2.withColumn()

In [ ]:
result2.show()

In [56]:
result2.filter(result2.dot > 0.010).show() 

PythonException: ignored

In [51]:
result2.coalesce(1)\
   .write\
   .format('com.databricks.spark.csv')\
   .options(header='true', delimiter='\t')\
   .save('drive/My Drive/arXiv/similarities_temp.csv')

In [52]:
result2.describe().show()

+-------+--------------------+--------------------+--------------------+
|summary|                   i|                   j|                 dot|
+-------+--------------------+--------------------+--------------------+
|  count|                  45|                  45|                  45|
|   mean|   704.0007422222224|   704.0014977777776|0.014873521698931068|
| stddev|4.525126261177885E-4|5.011300361538526E-4|0.023442189011643733|
|    min|           0704.0002|           0704.0004|1.797067603177058...|
|    max|           0704.0019|           0704.0022|  0.1420515225847642|
+-------+--------------------+--------------------+--------------------+



In [ ]:


dot_udf = F.udf(lambda x,y: float(x.dot(y)), DoubleType())

result_small = data.alias("i").join(data.alias("j"), F.col("i.ID") < F.col("j.ID"))\
    .select(
        F.col("i.ID").alias("i"), 
        F.col("j.ID").alias("j"), 
        dot_udf("i.norm", "j.norm").alias("dot"))\
    .sort("i", "j")

In [ ]:
result1.rdd.getNumPartitions()

KeyboardInterrupt: ignored

In [ ]:
result.coalesce(1)\
   .write\
   .format('com.databricks.spark.csv')\
   .options(header='true', delimiter='\t')\
   .save('drive/My Drive/arXiv/similarities.csv')

KeyboardInterrupt: ignored

In [ ]:
result.write.csv('/content/drive/My Drive/arXiv/cosine.csv')

KeyboardInterrupt: ignored

In [ ]:
result

KeyboardInterrupt: ignored

# Topic Modelling

In [ ]:
from pyspark.ml.clustering import LDA
num_topics = 6
max_iter = 10
lda = LDA(k=num_topics, 
          maxIter=max_iter, 
          featuresCol='tf_idf_features')
lda_model = lda.fit(tfidf_result)

vocab = tf_model.vocabulary
def get_words(token_list):
    return [vocab[token_id] for token_id in token_list]
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))

num_top_words = 7
topics = lda_model
     .describeTopics(num_top_words)
     .withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=100)

NameError: ignored